# Import

In [1]:
import pandas as pd 
import glob
import cv2 as cv
import random
import os

import matplotlib.pyplot as plt
import numpy as np
import random
from PIL import Image
import PIL.ImageOps    

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import torchvision.utils
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from sklearn.metrics import f1_score, accuracy_score

from tqdm.auto import tqdm
import timm
import math
from sklearn.model_selection import train_test_split

import segmentation_models_pytorch as smp
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Utils

In [2]:
# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = 'a'
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':30,
    'LEARNING_RATE':3e-4,
    # 'LEARNING_RATE':10,
    'BATCH_SIZE':8,
    'SEED':41
}

seed_everything(CFG['SEED']) # Seed 고정

In [3]:
#폴더 이동시 경로 수정이 필요할 수 있음 
train_source = glob.glob("../Data/train_source_image/*")
val_source = glob.glob("../Data/val_source_image/*")
train_gt = glob.glob("../Data/train_source_gt/*")
val_gt = glob.glob("../Data/val_source_gt/*")

train_source += val_source
train_gt += val_gt

# glob 이후에 정렬이 안되어 있기 때문에, source - gt matching을 위해 정렬
train_source.sort()
train_gt.sort()

print(train_source)

['../Data/train_source_image\\TRAIN_SOURCE_0000.png', '../Data/train_source_image\\TRAIN_SOURCE_0001.png', '../Data/train_source_image\\TRAIN_SOURCE_0002.png', '../Data/train_source_image\\TRAIN_SOURCE_0003.png', '../Data/train_source_image\\TRAIN_SOURCE_0004.png', '../Data/train_source_image\\TRAIN_SOURCE_0005.png', '../Data/train_source_image\\TRAIN_SOURCE_0006.png', '../Data/train_source_image\\TRAIN_SOURCE_0007.png', '../Data/train_source_image\\TRAIN_SOURCE_0008.png', '../Data/train_source_image\\TRAIN_SOURCE_0009.png', '../Data/train_source_image\\TRAIN_SOURCE_0010.png', '../Data/train_source_image\\TRAIN_SOURCE_0011.png', '../Data/train_source_image\\TRAIN_SOURCE_0012.png', '../Data/train_source_image\\TRAIN_SOURCE_0013.png', '../Data/train_source_image\\TRAIN_SOURCE_0014.png', '../Data/train_source_image\\TRAIN_SOURCE_0015.png', '../Data/train_source_image\\TRAIN_SOURCE_0016.png', '../Data/train_source_image\\TRAIN_SOURCE_0017.png', '../Data/train_source_image\\TRAIN_SOURCE_001

In [4]:
# DF 생성 
df = pd.DataFrame(columns=['source','gt'])
df['source'] = train_source
df['gt'] = train_gt
df

,source,gt
0,../Data/train_source_image\TRAIN_SOURCE_0000.png,../Data/train_source_gt\TRAIN_SOURCE_0000.png
1,../Data/train_source_image\TRAIN_SOURCE_0001.png,../Data/train_source_gt\TRAIN_SOURCE_0001.png
2,../Data/train_source_image\TRAIN_SOURCE_0002.png,../Data/train_source_gt\TRAIN_SOURCE_0002.png
3,../Data/train_source_image\TRAIN_SOURCE_0003.png,../Data/train_source_gt\TRAIN_SOURCE_0003.png
4,../Data/train_source_image\TRAIN_SOURCE_0004.png,../Data/train_source_gt\TRAIN_SOURCE_0004.png
...,...,...
2655,../Data/val_source_image\VALID_SOURCE_461.png,../Data/val_source_gt\VALID_SOURCE_461.png
2656,../Data/val_source_image\VALID_SOURCE_462.png,../Data/val_source_gt\VALID_SOURCE_462.png
2657,../Data/val_source_image\VALID_SOURCE_463.png,../Data/val_source_gt\VALID_SOURCE_463.png
2658,../Data/val_source_image\VALID_SOURCE_464.png,../Data/val_source_gt\VALID_SOURCE_464.png


# Custom Dataset

In [5]:
class CustomDataset(Dataset):
    def __init__(self, source, gt, transform=None, infer=False):
        self.source = source
        self.gt = gt
        self.transform = transform
        self.infer = infer


    def __getitem__(self, idx):
        img_path = self.source[idx]
        image = cv.imread(img_path)
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
        
        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image
        
        mask_path = self.gt[idx]
        mask = cv.imread(mask_path, cv.IMREAD_GRAYSCALE)
        mask[mask == 255] = 12 #/ 배경을 픽셀값 12로 간주 이거 원래 없던 값!

        if self.transform: # 알부네이션 먹이이는 형식으로 진행 
            augmented = self.transform(image=image, mask=mask) 
            image = augmented['image']
            mask = augmented['mask']
            
        return image, mask
    
    def __len__(self):
        return len(self.source)
    


# Transfrom - Data Augmentation

In [6]:
transform = A.Compose(
    [   
        A.Resize(224, 224),
        A.Normalize(),
        ToTensorV2()
    ]
)

# Data Loader

In [7]:
train, val, _, _ = train_test_split(df, _, test_size=0.2, random_state=CFG['SEED'])

In [8]:
train_dataset = CustomDataset(source = train['source'].values, gt = train['gt'].values, transform=transform, infer=False)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0)

val_dataset = CustomDataset(source = val['source'].values, gt = val['gt'].values, transform=transform, infer=False)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True, num_workers=0)

# Model

### SMP API

- model.encoder - pretrained backbone to extract features of different spatial resolution
- model.decoder - depends on models architecture (Unet/Linknet/PSPNet/FPN)
- model.segmentation_head - last block to produce required number of mask channels (include also optional upsampling and activation)
- model.classification_head - optional block which create classification head on top of encoder
- model.forward(x) - sequentially pass x through model`s encoder, decoder and segmentation head (and classification head if specified)

### Model Param
 - Docs - https://www.kaggle.com/code/ligtfeather/semantic-segmentation-is-easy-with-pytorch

In [9]:
aux_params=dict(
    pooling='max',             # one of 'avg', 'max'
    dropout=0.5,               # dropout ratio, default is None
    activation='sigmoid',      # activation function, default is None
    classes=13,                 # define number of output labels
)

# model = smp.Unet('mit_b5', encoder_weights='imagenet', classes=13, activation=None, encoder_depth=5, decoder_channels=[256, 128, 64, 32, 16] , aux_params=aux_params)
# model

In [10]:
import pytorch_lightning as pl
class DANN(pl.LightningModule):
    
    def __init__( self, arch, encoder_name, in_channels, out_classes, **kwargs) -> None:
        super().__init__()
        self.model = smp.create_model(
            arch, encoder_name=encoder_name, in_channels=in_channels, classes=out_classes, **kwargs
        )
        
        self.AdaptiveAvgPool2d = nn.AdaptiveAvgPool2d(output_size=1)
        self.Flatten = nn.Flatten(start_dim=1, end_dim=-1)
        self.Dropout = nn.Dropout(p=0.5, inplace=True)
        self.Classifier = nn.Linear(in_features=1280, out_features=2, bias=True)
        self.Activation = nn.Sigmoid()
        
    
    def forward(self, x):
        x = self.model.encoder(x)
        x = self.model.decoder(*x)
        x = self.model.segmentation_head(x)
        return x

c:\Users\EHmin\anaconda3\envs\EHmin\lib\site-packages\torchaudio\backend\utils.py:62: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [11]:
model = DANN("Unet", "mobilenet_v2", in_channels=3, out_classes=13)
model

DANN(
  (model): Unet(
    (encoder): MobileNetV2Encoder(
      (features): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): InvertedResidual(
          (conv): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): ReLU6(inplace=True)
            )
            (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
        )
        (2): InvertedResidual(
          (conv): Sequential(
            (0): Conv2dNor

In [12]:
# model = smp.Unet('mit_b5', encoder_weights='imagenet', classes=13, activation=None, encoder_depth=5, decoder_channels=[256, 128, 64, 32, 16] , aux_params=aux_params)
# model

# Validation

In [13]:
# define mIoU for Score >> 가져온 함수여서... batch 사이즈에 대한 고려가 안되어 있을 수 있음
def mIoU(pred_mask, mask, smooth=1e-10, n_classes=13):
    with torch.no_grad():
        pred_mask = F.softmax(pred_mask, dim=1)
        pred_mask = torch.argmax(pred_mask, dim=1)
        pred_mask = pred_mask.contiguous().view(-1)
        mask = mask.contiguous().view(-1)

        iou_per_class = []
        for clas in range(0, n_classes): #loop per pixel class
            true_class = pred_mask == clas
            true_label = mask == clas

            if true_label.long().sum().item() == 0: #no exist label in this loop
                iou_per_class.append(np.nan)
            else:
                intersect = torch.logical_and(true_class, true_label).sum().float().item()
                union = torch.logical_or(true_class, true_label).sum().float().item()

                iou = (intersect + smooth) / (union +smooth)
                iou_per_class.append(iou)
                
    return np.nanmean(iou_per_class)

In [14]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = 0
    val_score = 0
    with torch.no_grad():
        for source , gt in tqdm(iter(val_loader)):
            source = source.float().to(device)
            gt = gt.long().to(device)
            outputs = model(source)
            loss = criterion(outputs[0], gt.squeeze(1))
            val_loss += loss.item()
            val_score += mIoU(outputs[0], gt)
    
    return val_loss/len(val_loader) , val_score/len(val_loader)
    

# Train

In [15]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    # Model load 
    model = model.to(device) # 그냥 model.to(device)만 하면 저장 안됨

    # loss function과 optimizer 정의
    criterion = torch.nn.CrossEntropyLoss()
    # optimizer = torch.optim.Adam(model.parameters(), lr=CFG['LEARNING_RATE'])
    # 이거 밖에서 선언할거임 
    
    best_score = 0
    best_model = None
    
    for epoch in range(0, CFG['EPOCHS']):
        model.train()
        train_loss = 0
        
        for source , gt in tqdm(train_loader):
            source = source.float().to(device)
            gt = gt.long().to(device)
            
            optimizer.zero_grad() #! 이건 뭐해주는거지?? 추후에 확인 필
            outputs = model(source)
            outputs = torch.tensor(outputs)
            print(outputs)
            outputs.requires_grad_(True)
            
            loss = criterion(outputs, gt.squeeze(1))
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            
        _train_loss = train_loss/len(train_loader)
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val accuracy score : [{_val_score:.5f}]')
         
        if scheduler is not None:
            scheduler.step(_val_score)
        
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
            torch.save(best_model.state_dict(), "./models/Unet_mit_b5.pt")
    
    return best_model

In [16]:
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/133 [00:00<?, ?it/s]

C:\Users\EHmin\AppData\Local\Temp\ipykernel_13020\3319057227.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs = torch.tensor(outputs)


tensor([[[[ 2.2413e-01,  8.9117e-02, -8.2591e-02,  ...,  7.6416e-01,
            3.3901e-01, -3.5571e-01],
          [ 9.1192e-01,  2.9769e-01,  6.1533e-02,  ...,  3.6120e-01,
            8.6179e-01, -3.3792e-01],
          [ 7.2420e-01,  7.6226e-01,  6.3222e-01,  ...,  1.0477e-01,
            8.4451e-01, -1.9113e-01],
          ...,
          [ 2.1145e-01,  6.1998e-01,  7.0817e-02,  ...,  8.5769e-01,
            1.3099e+00,  1.5389e-01],
          [ 1.1312e-01,  3.4816e-01, -1.2586e-01,  ...,  2.8612e-01,
            1.1925e+00,  1.6211e-01],
          [ 7.8492e-02,  3.2841e-01,  4.1996e-01,  ...,  9.9867e-02,
            5.5861e-01, -6.3979e-04]],

         [[ 3.3420e-01, -5.3276e-01,  2.1967e-02,  ...,  2.6706e-01,
            1.0651e-01, -1.2096e-01],
          [ 5.8773e-01, -4.1777e-01, -1.8697e-01,  ..., -5.0068e-01,
           -6.6672e-01, -3.3243e-01],
          [ 4.6045e-01,  1.0145e-01, -1.1289e-01,  ..., -1.7552e-01,
           -8.5324e-01,  2.7121e-01],
          ...,
     

KeyboardInterrupt: 

# EVAL

In [ ]:
#폴더 이동시 경로 수정이 필요할 수 있음 
test_dataset = glob.glob("../Data/test_image/*")

# glob 이후에 정렬이 안되어 있기 때문에, source - gt matching을 위해 정렬
test_dataset.sort()

print(test_dataset)

In [ ]:
df_test = pd.DataFrame(columns=['test'])
df_test['test'] = test_dataset

df_test

In [ ]:
test_dataset = CustomDataset(source = df_test['test'].values ,gt = _ , transform=transform, infer=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=0 )

In [ ]:
model = smp.Unet('mit_b5', encoder_weights='imagenet', classes=13, activation=None, encoder_depth=5, decoder_channels=[256, 128, 64, 32, 16] , aux_params=aux_params)
model.load_state_dict(torch.load('./models/Unet_mit_b5.pt'))
model.to(device)

In [ ]:
with torch.no_grad():
    model.eval()
    result = []
    for images in tqdm(test_loader):
        images = images.float().to(device)
        outputs = model(images)
        outputs = torch.softmax(outputs[0], dim=1).cpu()
        outputs = torch.argmax(outputs, dim=1).numpy()
        # batch에 존재하는 각 이미지에 대해서 반복
        for pred in outputs:
            pred = pred.astype(np.uint8)
            pred = Image.fromarray(pred) # 이미지로 변환
            pred = pred.resize((960, 540), Image.NEAREST) # 960 x 540 사이즈로 변환
            pred = np.array(pred) # 다시 수치로 변환
            # class 0 ~ 11에 해당하는 경우에 마스크 형성 / 12(배경)는 제외하고 진행
            for class_id in range(12):
                class_mask = (pred == class_id).astype(np.uint8)
                if np.sum(class_mask) > 0: # 마스크가 존재하는 경우 encode
                    mask_rle = rle_encode(class_mask)
                    result.append(mask_rle)
                else: # 마스크가 존재하지 않는 경우 -1
                    result.append(-1)

# Submisssion

In [ ]:
submit = pd.read_csv('../Data/sample_submission.csv')
submit['mask_rle'] = result
submit

In [ ]:
submit.to_csv('./baseline_submit.csv', index=False)